In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [2]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/2022/본문_형태소분석/10월/나무문화_10월_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

398


In [3]:
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_10월_형태소분석.txt", encoding='UTF-8')
df

,split_str,org_idx,morphs,pos_tag
0,10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니 깜빡해버려서 몰아서 쓰는 중이다,0,"['10월', '1일', '날', '갔던', '캠핑', '인데', '포스팅', '을...","[('10월', 'Number'), ('1일', 'Number'), ('날', 'N..."
1,이날 날씨가 너무 좋다 보니 캠핑장과 펜션은 당연히 꽉 차버렸고 캠핑장 사장님 찬스...,0,"['이', '날', '날씨', '가', '너무', '좋다', '보니', '캠핑장',...","[('이', 'Determiner'), ('날', 'Noun'), ('날씨', 'N..."
2,친한 언니들 A언니와 B언니 부부와 함께 갔던 날 우리 부부까지 합치면 다섯 명 이...,0,"['친한', '언니', '들', 'A', '언니', '와', 'B', '언니', '...","[('친하다', 'Adjective'), ('언니', 'Noun'), ('들', '..."
3,푸릇푸릇 한 밤나무들이 인상적이다 주차장에도 차가 정 말 많았다,0,"['푸릇푸릇', '한', '밤나무', '들', '이', '인상', '적', '이다'...","[('푸릇푸릇', 'Noun'), ('하다', 'Verb'), ('밤나무', 'No..."
4,차가 많이 막혀서 일찍 출발했는데도 시간이 꽤 걸렸다,0,"['차가', '많이', '막혀서', '일찍', '출발', '했는데도', '시간', ...","[('차갑다', 'Adjective'), ('많이', 'Adverb'), ('막히다..."
...,...,...,...,...
1113264,관람을 마치고 나오면 야외에도 이렇게 좌불상들이 있다,24375,"['관람', '을', '마치', '고', '나오면', '야외', '에도', '이렇게...","[('관람', 'Noun'), ('을', 'Josa'), ('마치', 'Noun')..."
1113265,진짜 중국 스케일 최고 불교는 아니지만 중국 도시를 여행 할 때마다 사찰과 같은 불...,24375,"['진짜', '중국', '스케일', '최고', '불교', '는', '아니지만', '...","[('진짜', 'Noun'), ('중국', 'Noun'), ('스케일', 'Noun..."
1113266,뭔가 현실세계가 아닌 느낌이랄까,24375,"['뭔가', '현', '실세', '계', '가', '아닌', '느낌', '이', '...","[('뭔가', 'Noun'), ('현', 'Modifier'), ('실세', 'No..."
1113267,그래서 세계인이 주목하는 미래형 불교사찰인거겠지 이정도는 되어야 중국 3대 불교 성...,24375,"['그래서', '세계', '인', '이', '주목', '하는', '미래', '형',...","[('그래서', 'Adverb'), ('세계', 'Noun'), ('인', 'Suf..."


In [4]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs
0,10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니 깜빡해버려서 몰아서 쓰는 중이다,0,"['10월', '1일', '날', '갔던', '캠핑', '인데', '포스팅', '을...","[('10월', 'Number'), ('1일', 'Number'), ('날', 'N...",10월 1일 날 갔던 캠핑 인데 포스팅 을 미뤘더니 깜빡 해 버려서 몰아서 쓰는 중이 다
1,이날 날씨가 너무 좋다 보니 캠핑장과 펜션은 당연히 꽉 차버렸고 캠핑장 사장님 찬스...,0,"['이', '날', '날씨', '가', '너무', '좋다', '보니', '캠핑장',...","[('이', 'Determiner'), ('날', 'Noun'), ('날씨', 'N...",이 날 날씨 가 너무 좋다 보니 캠핑장 과 펜션 은 당연히 꽉 차 버렸고 캠핑장 사...
2,친한 언니들 A언니와 B언니 부부와 함께 갔던 날 우리 부부까지 합치면 다섯 명 이...,0,"['친한', '언니', '들', 'A', '언니', '와', 'B', '언니', '...","[('친하다', 'Adjective'), ('언니', 'Noun'), ('들', '...",친한 언니 들 A 언니 와 B 언니 부부 와 함께 갔던 날 우리 부부 까지 합치 면...
3,푸릇푸릇 한 밤나무들이 인상적이다 주차장에도 차가 정 말 많았다,0,"['푸릇푸릇', '한', '밤나무', '들', '이', '인상', '적', '이다'...","[('푸릇푸릇', 'Noun'), ('하다', 'Verb'), ('밤나무', 'No...",푸릇푸릇 한 밤나무 들 이 인상 적 이다 주차장 에도 차가 정 말 많았다
4,차가 많이 막혀서 일찍 출발했는데도 시간이 꽤 걸렸다,0,"['차가', '많이', '막혀서', '일찍', '출발', '했는데도', '시간', ...","[('차갑다', 'Adjective'), ('많이', 'Adverb'), ('막히다...",차가 많이 막혀서 일찍 출발 했는데도 시간 이 꽤 걸렸다
...,...,...,...,...,...
1113264,관람을 마치고 나오면 야외에도 이렇게 좌불상들이 있다,24375,"['관람', '을', '마치', '고', '나오면', '야외', '에도', '이렇게...","[('관람', 'Noun'), ('을', 'Josa'), ('마치', 'Noun')...",관람 을 마치 고 나오면 야외 에도 이렇게 좌 불상 들 이 있다
1113265,진짜 중국 스케일 최고 불교는 아니지만 중국 도시를 여행 할 때마다 사찰과 같은 불...,24375,"['진짜', '중국', '스케일', '최고', '불교', '는', '아니지만', '...","[('진짜', 'Noun'), ('중국', 'Noun'), ('스케일', 'Noun...",진짜 중국 스케일 최고 불교 는 아니지만 중국 도시 를 여행 할 때 마다 사찰 과 ...
1113266,뭔가 현실세계가 아닌 느낌이랄까,24375,"['뭔가', '현', '실세', '계', '가', '아닌', '느낌', '이', '...","[('뭔가', 'Noun'), ('현', 'Modifier'), ('실세', 'No...",뭔가 현 실세 계 가 아닌 느낌 이 랄 까
1113267,그래서 세계인이 주목하는 미래형 불교사찰인거겠지 이정도는 되어야 중국 3대 불교 성...,24375,"['그래서', '세계', '인', '이', '주목', '하는', '미래', '형',...","[('그래서', 'Adverb'), ('세계', 'Noun'), ('인', 'Suf...",그래서 세계 인 이 주목 하는 미래 형 불교 사찰 인거 겠지 이 정도 는 되어야 중...


In [5]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs,joined_sentences
0,10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니 깜빡해버려서 몰아서 쓰는 중이다,0,"['10월', '1일', '날', '갔던', '캠핑', '인데', '포스팅', '을...","[('10월', 'Number'), ('1일', 'Number'), ('날', 'N...",10월 1일 날 갔던 캠핑 인데 포스팅 을 미뤘더니 깜빡 해 버려서 몰아서 쓰는 중이 다,10월 1일 날 갔던 캠핑 인데 포스팅 을 미뤘더니 깜빡 해 버려서 몰아서 쓰는 중...
1,이날 날씨가 너무 좋다 보니 캠핑장과 펜션은 당연히 꽉 차버렸고 캠핑장 사장님 찬스...,0,"['이', '날', '날씨', '가', '너무', '좋다', '보니', '캠핑장',...","[('이', 'Determiner'), ('날', 'Noun'), ('날씨', 'N...",이 날 날씨 가 너무 좋다 보니 캠핑장 과 펜션 은 당연히 꽉 차 버렸고 캠핑장 사...,10월 1일 날 갔던 캠핑 인데 포스팅 을 미뤘더니 깜빡 해 버려서 몰아서 쓰는 중...
2,친한 언니들 A언니와 B언니 부부와 함께 갔던 날 우리 부부까지 합치면 다섯 명 이...,0,"['친한', '언니', '들', 'A', '언니', '와', 'B', '언니', '...","[('친하다', 'Adjective'), ('언니', 'Noun'), ('들', '...",친한 언니 들 A 언니 와 B 언니 부부 와 함께 갔던 날 우리 부부 까지 합치 면...,10월 1일 날 갔던 캠핑 인데 포스팅 을 미뤘더니 깜빡 해 버려서 몰아서 쓰는 중...
3,푸릇푸릇 한 밤나무들이 인상적이다 주차장에도 차가 정 말 많았다,0,"['푸릇푸릇', '한', '밤나무', '들', '이', '인상', '적', '이다'...","[('푸릇푸릇', 'Noun'), ('하다', 'Verb'), ('밤나무', 'No...",푸릇푸릇 한 밤나무 들 이 인상 적 이다 주차장 에도 차가 정 말 많았다,10월 1일 날 갔던 캠핑 인데 포스팅 을 미뤘더니 깜빡 해 버려서 몰아서 쓰는 중...
4,차가 많이 막혀서 일찍 출발했는데도 시간이 꽤 걸렸다,0,"['차가', '많이', '막혀서', '일찍', '출발', '했는데도', '시간', ...","[('차갑다', 'Adjective'), ('많이', 'Adverb'), ('막히다...",차가 많이 막혀서 일찍 출발 했는데도 시간 이 꽤 걸렸다,10월 1일 날 갔던 캠핑 인데 포스팅 을 미뤘더니 깜빡 해 버려서 몰아서 쓰는 중...
...,...,...,...,...,...,...
1113264,관람을 마치고 나오면 야외에도 이렇게 좌불상들이 있다,24375,"['관람', '을', '마치', '고', '나오면', '야외', '에도', '이렇게...","[('관람', 'Noun'), ('을', 'Josa'), ('마치', 'Noun')...",관람 을 마치 고 나오면 야외 에도 이렇게 좌 불상 들 이 있다,2일 차 일정 난징대학살 기념관 l 우수 산 불 정궁 불정탑 우수 산 주소 18 관...
1113265,진짜 중국 스케일 최고 불교는 아니지만 중국 도시를 여행 할 때마다 사찰과 같은 불...,24375,"['진짜', '중국', '스케일', '최고', '불교', '는', '아니지만', '...","[('진짜', 'Noun'), ('중국', 'Noun'), ('스케일', 'Noun...",진짜 중국 스케일 최고 불교 는 아니지만 중국 도시 를 여행 할 때 마다 사찰 과 ...,2일 차 일정 난징대학살 기념관 l 우수 산 불 정궁 불정탑 우수 산 주소 18 관...
1113266,뭔가 현실세계가 아닌 느낌이랄까,24375,"['뭔가', '현', '실세', '계', '가', '아닌', '느낌', '이', '...","[('뭔가', 'Noun'), ('현', 'Modifier'), ('실세', 'No...",뭔가 현 실세 계 가 아닌 느낌 이 랄 까,2일 차 일정 난징대학살 기념관 l 우수 산 불 정궁 불정탑 우수 산 주소 18 관...
1113267,그래서 세계인이 주목하는 미래형 불교사찰인거겠지 이정도는 되어야 중국 3대 불교 성...,24375,"['그래서', '세계', '인', '이', '주목', '하는', '미래', '형',...","[('그래서', 'Adverb'), ('세계', 'Noun'), ('인', 'Suf...",그래서 세계 인 이 주목 하는 미래 형 불교 사찰 인거 겠지 이 정도 는 되어야 중...,2일 차 일정 난징대학살 기념관 l 우수 산 불 정궁 불정탑 우수 산 주소 18 관...


In [6]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,10월 1일 날 갔던 캠핑 인데 포스팅 을 미뤘더니 깜빡 해 버려서 몰아서 쓰는 중...
1,1,주일 아침 기상 알람 이 울 리기 전 아빠 에게 로부터 전화 가 왔다 어젯밤 들어오...
2,2,San Francisco CA United States of America 안녕하세...
3,3,세월 은 유수 와 같다고 하더니 2022년 을 시작 한 게 엊그제 같은데 벌써 10...
4,4,저번 에 리움 을 방문 했을 때 에 시간 이 모자라 기획전 을 관람 하지 못 했다 ...
...,...,...
24366,24371,DAY 2 2022 07 03 시차 로 인해 둘쨋 날 이지만 3일 입니다요 장거리 ...
24367,24372,카이로 3일 차 모마 시티투어 빡세 다 아침 부터 오후 4 5시 까지 일정 이 빽빽...
24368,24373,가끔 줄 여서 노바 NoVA 라고 부르는 워싱턴 DC 인근 의 북부 버지니아 Nor...
24369,24374,블로그 busan diary 2022 37 개 의 글 busan diary 2022...


In [7]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [8]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [9]:
tdm.shape

(24371, 500292)

In [10]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0222797 , 0.01003441, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02889895, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,...,500282,500283,500284,500285,500286,500287,500288,500289,500290,500291
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.018717,0.033720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.072366,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24366,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24367,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24368,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24369,0.022280,0.010034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [13]:
tfidf_DF

,0,00,000,0001,00010001,0001009981,0001177862,0001178223,00012,0001226873,...,힝구,힝기스,힝둥새,힝제,힝터,힝호,힝힝,힠,힣,힣힣
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.018717,0.033720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.072366,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24366,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24367,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24368,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24369,0.022280,0.010034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
tfidf_DF.to_csv("../dataset/2022/본문_TF-IDF/10월/나무문화_10월_TF-IDF행렬(원본).csv", index=False, encoding='UTF-8')

In [17]:
import re
# 데이터프레임에서 한글로만 이루어진 column 필터링을 위한 정규표현식 패턴
pattern = r'^[ㄱ-ㅎㅏ-ㅣ가-힣\s]+$'

# 한글로만 이루어진 column으로 된 데이터프레임 생성
korean_df = tfidf_DF[[column for column in tfidf_DF.columns if re.match(pattern, column)]]

print(korean_df)

              ㄱ   ㄱㄱ  ㄱㄱㄱ  ㄱㄱㄷㄹㅋㅍㄷㅈㅁㅈ  ㄱㄱㅅ  ㄱㄱㅆ  ㄱㄱㅋㄱㅋㅋ  ㄱㄱㅑ   ㄱㄴ  ㄱㄴㄷ  ...  \
0      0.000000  0.0  0.0         0.0  0.0  0.0     0.0  0.0  0.0  0.0  ...   
1      0.000000  0.0  0.0         0.0  0.0  0.0     0.0  0.0  0.0  0.0  ...   
2      0.000000  0.0  0.0         0.0  0.0  0.0     0.0  0.0  0.0  0.0  ...   
3      0.000000  0.0  0.0         0.0  0.0  0.0     0.0  0.0  0.0  0.0  ...   
4      0.000000  0.0  0.0         0.0  0.0  0.0     0.0  0.0  0.0  0.0  ...   
...         ...  ...  ...         ...  ...  ...     ...  ...  ...  ...  ...   
24366  0.016766  0.0  0.0         0.0  0.0  0.0     0.0  0.0  0.0  0.0  ...   
24367  0.000000  0.0  0.0         0.0  0.0  0.0     0.0  0.0  0.0  0.0  ...   
24368  0.000000  0.0  0.0         0.0  0.0  0.0     0.0  0.0  0.0  0.0  ...   
24369  0.000000  0.0  0.0         0.0  0.0  0.0     0.0  0.0  0.0  0.0  ...   
24370  0.000000  0.0  0.0         0.0  0.0  0.0     0.0  0.0  0.0  0.0  ...   

        힝구  힝기스  힝둥새   힝제   힝터   힝호   힝힝    힠    힣 

In [18]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,135.280251
1,00,370.449346
2,000,47.352479
3,0001,0.674531
4,00010001,0.059091
...,...,...
500287,힝호,0.014812
500288,힝힝,0.329585
500289,힠,0.066283
500290,힣,0.737909


In [19]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/2022/본문_TF-IDF/10월/나무문화_10월_TF-IDF행렬합.txt", index=False, encoding='UTF-8')